# RNN, LSTM, GRU을 이용한 텍스트 생성
- 참고: http://solarisailab.com/archives/1451, https://aikorea.org/blog/rnn-tutorial-3/, https://excelsior-cjh.tistory.com/185, https://dgkim5360.tistory.com/entry/understanding-long-short-term-memory-lstm-kr
<pre>
'경마장에 있는 말이 뛰고 있다‘
'그의 말이 법이다‘
'가는 말이 고와야 오는 말이 곱다'
</pre>

In [1]:
import numpy as np
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
text = '''경마장에 있는 말이 뛰고 있다.
그의 말이 법이다.
가는 말이 고와야 오는 말이 곱다.'''

- 텍스트 전처리

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [4]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts([text])  

In [5]:
# 특징
# 1. word_index 순서가 가나다 순서가 아니고, 많이 나온 단어 순
# 2. 0 index가 없다.
# 참고: https://codetorial.net/tensorflow/natural_language_processing_in_tensorflow_01.html
t.word_index

{'가는': 8,
 '경마장에': 2,
 '고와야': 9,
 '곱다': 11,
 '그의': 6,
 '뛰고': 4,
 '말이': 1,
 '법이다': 7,
 '오는': 10,
 '있는': 3,
 '있다': 5}

In [6]:
# 단어 집합 크기 - Keras Tokenizer의 시작 인덱스가 1이기 때문
vocab_size = len(t.word_index) + 1
vocab_size

12

In [7]:
# 참고: https://han-py.tistory.com/284
t.texts_to_sequences(['그의 말이 법이다.'])

[[6, 1, 7]]

In [8]:
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    print(encoded)

[2, 3, 1, 4, 5]
[6, 1, 7]
[8, 1, 9, 10, 1, 11]


In [9]:
sequences = []
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequences.append(encoded[:i+1])

sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [10]:
max_len = max(len(seq) for seq in sequences)
max_len

6

- Padding
    - 전체 샘플 길이를 max_len에 맞추도록 0를 추가
    - pre 옵션을 주면 앞쪽을 채움

In [11]:
# RNN의 input_shape(timesteps => 5개 , input_dim => 12개의 단어를 Embedding)
# 참고: https://velog.io/@ganta/%ED%8C%A8%EB%94%A9Padding
#   - https://simpling.tistory.com/entry/Embedding-%EC%9D%B4%EB%9E%80-%EB%AC%B4%EC%97%87%EC%9D%B8%EA%B0%80-%EC%9D%B4%ED%95%B4%ED%95%98%EA%B8%B0
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences[:5]

array([[0, 0, 0, 0, 2, 3],
       [0, 0, 0, 2, 3, 1],
       [0, 0, 2, 3, 1, 4],
       [0, 2, 3, 1, 4, 5],
       [0, 0, 0, 0, 6, 1]], dtype=int32)

In [12]:
X = sequences[:, :-1]
y = sequences[:, -1]

In [13]:
X.shape

(11, 5)

In [14]:
Y = to_categorical(y)
Y[:3]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

### 모델 정의
- Embedding
- SimpleRNN, LSTM, GRU
- Dense

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.layers import LSTM, GRU

- 모델 검증

In [17]:
from google.colab import files
up = files.upload()

Saving my_util.py to my_util.py


In [18]:
from my_util import sentence_generation

### 모델 변화
- SimpleRNN, GRU, LSTM
- Embedding vector 갯수: 2, 4
- RNN 유닛 갯수: 24, 32

In [23]:
def execute_RNN_model(n_embed, n_unit):
    model = Sequential([ 
        Embedding(vocab_size, n_embed, input_length=max_len-1),
        SimpleRNN(n_unit),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
    hist = model.fit(X, Y, epochs=200, verbose=0)
    print('=' * 50)
    print(f"정확도: {hist.history['accuracy'][-1]}")
    print(sentence_generation(model, t, max_len, '경마장에', 4))
    print(sentence_generation(model, t, max_len, '그의', 2))
    print(sentence_generation(model, t, max_len, '가는', 5))
    print()

In [24]:
def execute_LSTM_model(n_embed, n_unit):
    model = Sequential([ 
        Embedding(vocab_size, n_embed, input_length=max_len-1),
        LSTM(n_unit),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
    hist = model.fit(X, Y, epochs=200, verbose=0)
    print('=' * 50)
    print(f"정확도: {hist.history['accuracy'][-1]}")
    print(sentence_generation(model, t, max_len, '경마장에', 4))
    print(sentence_generation(model, t, max_len, '그의', 2))
    print(sentence_generation(model, t, max_len, '가는', 5))
    print()

In [25]:
def execute_GRU_model(n_embed, n_unit):
    model = Sequential([ 
        Embedding(vocab_size, n_embed, input_length=max_len-1),
        GRU(n_unit),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
    hist = model.fit(X, Y, epochs=200, verbose=0)
    print('=' * 50)
    print(f"정확도: {hist.history['accuracy'][-1]}")
    print(sentence_generation(model, t, max_len, '경마장에', 4))
    print(sentence_generation(model, t, max_len, '그의', 2))
    print(sentence_generation(model, t, max_len, '가는', 5))
    print()

LSTM, GRU - 긴 문장일 때 유용하다.

In [26]:
for n_embed in [2,4]:
    for n_unit in [24, 32]:
        print(f'SimpleRNN, {n_embed}, {n_unit} ====================')
        execute_RNN_model(n_embed, n_unit)
        print()
        print(f'LSTM, {n_embed}, {n_unit} ====================')
        execute_LSTM_model(n_embed, n_unit)
        print()
        print(f'GRU, {n_embed}, {n_unit} ====================')
        execute_GRU_model(n_embed, n_unit)
        print()

SimpleRNN, 2, 24 ====================
정확도: 0.7272727489471436
경마장에 말이 말이 오는 말이
그의 말이 말이
가는 말이 말이 오는 말이 곱다


LSTM, 2, 24 ====================
정확도: 0.4545454680919647
경마장에 말이 말이 말이 말이
그의 말이 말이
가는 말이 말이 말이 말이 말이


GRU, 2, 24 ====================
정확도: 0.4545454680919647
경마장에 말이 말이 말이 곱다
그의 말이 말이
가는 말이 말이 오는 곱다 곱다


SimpleRNN, 2, 32 ====================
정확도: 0.6363636255264282
경마장에 말이 말이 뛰고 있다
그의 말이 말이
가는 말이 말이 말이 말이 곱다


LSTM, 2, 32 ====================
정확도: 0.6363636255264282
경마장에 말이 말이 뛰고 있다
그의 말이 말이
가는 말이 말이 말이 뛰고 있다


GRU, 2, 32 ====================
정확도: 1.0
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다


SimpleRNN, 4, 24 ====================
정확도: 1.0
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다


LSTM, 4, 24 ====================
정확도: 0.5454545617103577
경마장에 말이 말이 뛰고 곱다
그의 말이 말이
가는 말이 말이 말이 뛰고 곱다


GRU, 4, 24 ====================
정확도: 0.8181818127632141
경마장에 말이 고와야 오는 말이
그의 말이 고와야
가는 말이 고와야 오는 말이 곱다


SimpleRNN, 4, 32 ====================
정확도: 0.9090909361839294
경마장에 말이 말이 뛰고 있다
그의 말이